In [12]:
import numpy as np
import os
np.random.seed(42)

In [13]:

# Create arrays with filenames and lables
inFile = open("C:/Users/Lukas/Desktop/commaai-speed-challenge/data/train-grouped.txt", "r")

all_images = {}
all_labels = {}
# exclude last group (30mph) beacuse there is so few 
for i in range(6):
    all_images[i] = []
for i, v in enumerate(inFile):
    if int(v)//5 < 6:
        #add the ith image to its group
        all_images[int(v)//5].append(str(i)+".jpg")
        all_labels[str(i)+".jpg"] = int(v)//5
    
min_size = int(1e9)
for i in range(6):
    min_size = min(len(all_images[i]), min_size)
    np_arr = np.array(all_images[i])
    all_images[i] = np_arr
    print(np_arr.shape)

print(min_size)


(2496,)
(2597,)
(4553,)
(3141,)
(2015,)
(4820,)
2015


In [14]:
train_images = {}
val_images = {}
test_images = {}
for i in range(6):
    np.random.shuffle(all_images[i])
    train_images[i] = all_images[i][:1400]
    val_images[i] = all_images[i][1400:1700]
    test_images[i] = all_images[i][1700:2000]
    print(i, "mph group dataset: ", train_images[i].shape, val_images[i].shape, test_images[i].shape)

['9316.jpg' '9317.jpg' '9318.jpg' ... '19691.jpg' '19692.jpg' '19693.jpg']
['18880.jpg' '17981.jpg' '15632.jpg' ... '15911.jpg' '16493.jpg'
 '15590.jpg']
0 mph group dataset:  (1400,) (300,) (300,)
['9285.jpg' '9286.jpg' '9287.jpg' ... '20397.jpg' '20398.jpg' '20399.jpg']
['14737.jpg' '16048.jpg' '16033.jpg' ... '18125.jpg' '19789.jpg'
 '19326.jpg']
1 mph group dataset:  (1400,) (300,) (300,)
['8848.jpg' '8849.jpg' '8850.jpg' ... '20308.jpg' '20309.jpg' '20310.jpg']
['14309.jpg' '10395.jpg' '19896.jpg' ... '9679.jpg' '8962.jpg' '11800.jpg']
2 mph group dataset:  (1400,) (300,) (300,)
['1266.jpg' '1272.jpg' '1273.jpg' ... '20039.jpg' '20040.jpg' '20041.jpg']
['18402.jpg' '6055.jpg' '17705.jpg' ... '8522.jpg' '17533.jpg' '8600.jpg']
3 mph group dataset:  (1400,) (300,) (300,)
['292.jpg' '293.jpg' '294.jpg' ... '8510.jpg' '8511.jpg' '8512.jpg']
['976.jpg' '1022.jpg' '1143.jpg' ... '990.jpg' '5803.jpg' '1087.jpg']
4 mph group dataset:  (1400,) (300,) (300,)
['178.jpg' '179.jpg' '180.jpg' .

In [10]:
train_images_all = np.concatenate([train_images[x] for x in train_images])
val_images_all = np.concatenate([val_images[x] for x in val_images])
test_images_all = np.concatenate([test_images[x] for x in test_images])

train_images_all.shape, val_images_all.shape

((8400,), (1800,))

In [11]:
s = np.concatenate([train_images_all] + [val_images_all] + [test_images_all])
print(len(s))
s = set(s)
print(len(s))
if len(train_images_all)+len(val_images_all)+len(test_images_all) != len(s):
    print("Datasets has crossover")
else:
    print("Dataset is good")

12000
12000
Dataset is good


In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
%matplotlib inline

In [ ]:
IMG_DIM = (150, 150)

train_images_array = [img_to_array(load_img("C:/Users/Lukas/Desktop/commaai-speed-challenge/data/frames/"+img, target_size=IMG_DIM)) for img in train_images_all]
train_images_array = np.array(train_images_array)

val_images_array = [img_to_array(load_img("C:/Users/Lukas/Desktop/commaai-speed-challenge/data/frames/"+img, target_size=IMG_DIM)) for img in val_images_all]
val_images_array = np.array(val_images_array)

print('Train dataset shape:', train_images_array.shape, 
      '\tValidation dataset shape:', val_images_array.shape)


In [ ]:
train_labels = []
for fn in train_images_all:
    train_labels.append(all_labels[fn])
    
val_labels = []
for fn in val_images_all:
    val_labels.append(all_labels[fn])

print("labels length: ", len(train_labels), len(val_labels))

In [ ]:
# encode text category labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
validation_labels_enc = le.transform(val_labels)

validation_labels_enc

In [ ]:
train_imgs_scaled = train_images_array.astype('float32')
validation_imgs_scaled  = val_images_array.astype('float32')
train_imgs_scaled /= 255
validation_imgs_scaled /= 255

print(train_images_array[0].shape)
array_to_img(train_images_array[0])

In [ ]:
from keras.applications import vgg16
from keras.models import Model
import keras
import pandas as pd

In [ ]:
#25 devides both 1400 and 300
batch_size = 25
#remember, we excluded the last class
num_classes = 6
epochs = 20
input_shape = (150, 150, 3)

In [ ]:
vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

In [ ]:
output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)


In [ ]:
vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])    

In [ ]:
bottleneck_feature_example = vgg.predict(train_imgs_scaled[0:1])
print(bottleneck_feature_example.shape)
plt.imshow(bottleneck_feature_example[0][:,:,0])

In [ ]:
def get_bottleneck_features(model, input_imgs):
    features = model.predict(input_imgs, verbose=1)
    return features
    
train_features_vgg = get_bottleneck_features(vgg_model, train_imgs_scaled)
validation_features_vgg = get_bottleneck_features(vgg_model, validation_imgs_scaled)

print('Train Bottleneck Features:', train_features_vgg.shape, 
      '\tValidation Bottleneck Features:', validation_features_vgg.shape)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers

In [ ]:
input_shape = vgg_model.output_shape[1]
input_shape

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(input_shape,)))
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='sigmoid'))


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x=train_features_vgg, y=train_labels_enc,
                    validation_data=(validation_features_vgg, validation_labels_enc),
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

In [ ]:
model.save('model_classify_frames_v2.h5')

In [ ]:
#plot results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_images_array = [img_to_array(load_img("C:/Users/Lukas/Desktop/commaai-speed-challenge/data/frames/"+img, target_size=IMG_DIM)) for img in test_images_all]
test_images_array = np.array(test_images_array)

test_imgs_scaled = test_images_array.astype('float32')
test_imgs_scaled /= 255

print(test_images_array[0].shape)
array_to_img(test_images_array[0])

In [ ]:
test_bottleneck_features = get_bottleneck_features(vgg_model, test_imgs_scaled)


In [ ]:
predicted_ids = model.predict_classes(test_bottleneck_features, verbose=1)
predicted_class_names = [i * 5 for i in predicted_ids]


In [ ]:
test_labels = []
for fn in test_images_all:
    test_labels.append(all_labels[fn])


In [ ]:
results = model.evaluate(test_bottleneck_features, np.array(test_labels), batch_size=batch_size)
